# Installing Semantic Kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.7.1-alpha"

#!import Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.7.1-alpha

In [6]:
#!import Secrets.cs

#pragma warning disable SKEXP0050

var bingConnector = new BingConnector(Secrets.BingSearchKey);
var bing = new WebSearchEnginePlugin(bingConnector);

kernel.ImportPluginFromObject(bing, "bing");

### 📋 Add some arguments

In [7]:
var promptTemplate = "When is the next total {{$input}} eclipse?";
var function = kernel.Plugins["bing"]["search"];
var bingResult = await kernel.InvokeAsync(function, new() { ["query"] = promptTemplate });

var currentContext = bingResult.GetValue<string>();
display(currentContext)

["Eclipses. Next Eclipses in the Netherlands. Partial Lunar Eclipse. Sep 18, 2024. Partial Lunar Eclipse. Mar 14, 2025. Partial Solar Eclipse. Mar 29, 2025. Total Lunar Eclipse. Sep 7, 2025. Partial Solar Eclipse. Aug 12, 2026. Partial Lunar Eclipse. Aug 28, 2026. Penumbral Lunar Eclipse. Feb 20, 2027. Partial Solar Eclipse. Aug 2, 2027.","Astronomy Index. List of Total Solar Eclipses Worldwide Next 10 Years. Check where the eclipses are visible and if you can see them.","The Monday, April 8, 2024, total solar eclipse will cross North America, passing over Mexico, the United States, and Canada. The total solar eclipse will begin over the South Pacific Ocean.","It’s never too soon to start thinking about the next solar eclipse. On April 8, the moon’s shadow will sweep across North America, bringing a total solar eclipse over the homes of more than 30...","When will the next totality be visible from the U.S.? The next U.S. taste of totality comes in 2033 when an eclipse brushes Alaska an

### 🏃‍♀️ Run prompt

In [ ]:
var promptTemplate = """
    When is the next total {{$input}} eclipse?

    ### real-time search result context
    {{$currentContext}} 

    Respond only with JSON.
    """;

var executionSettings = new OpenAIPromptExecutionSettings 
{
     MaxTokens = 500,
     Temperature = 0.1 
};
var kernelArguments = new KernelArguments(executionSettings)
{
    ["input"] = "solar",
    ["currentContext"] = currentContext
};

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);
var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response.GetValue<string>())

### 👀 See rendered prompt

In [ ]:
var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplateRenderer = promptTemplateFactory.Create(new PromptTemplateConfig(promptTemplate));
var renderedPrompt = await promptTemplateRenderer.RenderAsync(kernel, kernelArguments);

display(renderedPrompt)